In [3]:
import pandas as pd
df = pd.read_csv('directors-movies.csv')

movie_overviews = df["overview"].tolist()

In [4]:
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv(".env")

embeddings = OpenAIEmbeddings()

embeddings_list = []

for overview in movie_overviews[0:80]:
  embeddings_list.append(embeddings.embed_query(overview))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
import numpy as np

embeddings_list = np.array(embeddings_list).astype("float32")

In [6]:
import faiss

index = faiss.IndexFlatL2(1536)
index.add(embeddings_list)

In [7]:
faiss.write_index(index, "embeddings.index")

In [8]:
index = faiss.read_index("embeddings.index")

In [9]:
test_sample = embeddings.embed_query("When teenager Elle's first kiss leads to a forbidden romance with the hottest boy in high school, she risks her relationship with her best friend.")

test_sample = np.array([test_sample]).astype("float32")

In [10]:
k = 5
distances, indices = index.search(test_sample, k)

In [13]:
np.array(movie_overviews)[indices[0]]

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [ ]:
df.iloc[indices[0]]